In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
s3_access_key = os.environ.get("S3_ACCESS_KEY")
s3_secret_key = os.environ.get("S3_SECRET_KEY")
CORES = 2 # Remember to use 2 cores for laptop work and 4 cores for local machine

In [ ]:
Common columns
Different columns
Compare data for common columns in both files
Difference in the amount of items stored in each parquet file
Count and group the occurrences of the value “trustRate” in each file
Count and group the occurrences of the value “humanId” in each file
Determine based on the contents of the compared files whether there is a major or critical difference in terms of data
Any additional statistical information considered useful is appreciated (within the context of an identity resolution process)


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Loyalty") \
    .config("spark.master", f"local[{CORES}]") \
    .config("spark.executor.cores", f"{CORES}") \
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.3.1") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", s3_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret_key) \
    .getOrCreate()

In [3]:
#Fetch part001 parquet data from S3 
df_part001 = spark.read.parquet("s3a://data-test-202302/adstra/part001.parquet")
df_part001.createOrReplaceTempView("df_part001")
new_df = spark.sql("SELECT * FROM df_part001 OFFSET 1")
df_part001 = new_df.toDF(*list(df_part001.first()))
df_part001.show(5)

+--------+------+------+--------------------+--------+-------+-----+
|   index| fname| lname|               email|socialid|country|state|
+--------+------+------+--------------------+--------+-------+-----+
|r0000001| james|robles|    person1@mail.net|774121t9|     mx|   tj|
|r0000002|daniel| boyle|roddaniel445@comp...|998112a4|     ca|   on|
|r0000003|  xxxx| xxxxx|              xxxxxx|878445b2|     us|   fl|
|r0000004| marie| velez|marie.rodarte123@...|798125x4|     ca|   ot|
|r0000005|MARTIN|TORRES|not.provided@abc.com|981210o2|     us|   ca|
+--------+------+------+--------------------+--------+-------+-----+
only showing top 5 rows



In [8]:
#Fetch results A, B and C parquets data from S3 
df_results_a = spark.read.parquet("s3a://data-test-202302/adstra/resultsA.parquet")
df_results_b = spark.read.parquet("s3a://data-test-202302/adstra/resultsB.parquet")
df_results_c = spark.read.parquet("s3a://data-test-202302/adstra/resultsC.parquet")